In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json, time, os, sys, glob
import shutil
import warnings
import numpy as np
import pandas as pd
import torch
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split, Subset
import copy
import torch.nn as nn
import torch.nn.functional as F
import random
import os.path
import subprocess
from tqdm import tqdm
from omegaconf import OmegaConf
import pytorch_lightning as pl
from terrace.batch import DataLoader
from typing import Sequence

from fireprot_dataset import *
from training.model_utils import featurize, loss_smoothed, loss_nll, get_std_opt, ProteinMPNN
from training.utils import worker_init_fn, get_pdbs, loader_pdb, build_training_clusters, PDB_dataset, StructureDataset, StructureLoader
from protein_mpnn_utils import loss_nll, loss_smoothed, gather_edges, gather_nodes, gather_nodes_t, cat_neighbors_nodes, _scores, _S_to_seq, tied_featurize, parse_PDB
from protein_mpnn_utils import StructureDataset, StructureDatasetPDB, ProteinMPNN
from kaggle_dataset import KaggleTrainDataset
from transfer_model import *
from train import *
from combo_dataset import *

In [3]:
cfg = OmegaConf.load("config.yaml")
cfg = OmegaConf.merge(cfg, OmegaConf.load("local.yaml"))

In [5]:
d = MegaScaleDataset(cfg, "val")

/home/boris/miniconda3/envs/chem/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3377: DtypeWarning: Columns (30,36) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
d.df.deltaG

0         4.091166
1         4.093463
2         3.938306
3         4.051388
4         3.965257
            ...   
851547   -3.148557
851548   -3.138319
851549   -2.567302
851550   -2.129308
851551   -2.245312
Name: deltaG, Length: 851552, dtype: float64